In [16]:
import tensorflow as tf
import numpy as np

print("Okay.")

Okay.


In [25]:
from random import shuffle
import sys

def func_test(p,q):
    return float(p)**2+float(q)

def func(p,q):
    p = float(p)
    q = float(q)
    return 0.5*np.log(4*p**2-3*q**2)

def model(num_param,batch_size):
    with tf.variable_scope("pq", reuse=True):
        input_p = tf.placeholder(name="input_param", dtype=tf.float32, shape=(num_param,batch_size))
        input_volume = tf.placeholder(name="input_volume", dtype=tf.float32, shape=(batch_size))
        input_volume = tf.expand_dims(input_volume,0)
        p = input_p[0]
        q = input_p[1]
        print("input p:",input_p.get_shape().as_list())
        print("input volume:",input_volume.get_shape().as_list())
        print("p:",p.get_shape().as_list())
        print("q:",q.get_shape().as_list())
        # variable 1:
#     with tf.variable_scope("p", reuse=True):
        const_p = tf.Variable(tf.ones(shape=(), dtype=tf.int32), name="const_p_sq")
        const_p = tf.sqrt(tf.square(tf.to_float(tf.to_float(const_p))))
        c_p = tf.scalar_mul(tf.to_float(const_p),p)
        p_sq = tf.square(p,name="p_sq")
        const_p_sq = tf.Variable(tf.ones(shape=(), dtype=tf.int32), name="const_p_sq")
        const_p_sq = tf.sqrt(tf.square(tf.to_float(tf.to_float(const_p_sq))))
        c_p_sq = tf.scalar_mul(tf.to_float(const_p_sq),p_sq)
        p_sum = tf.math.add(c_p,c_p_sq)
        p_1 = tf.stack([c_p,c_p_sq,p_sum])
        print("p1:",p_1.get_shape().as_list())
        # variable 2:
#     with tf.variable_scope("q", reuse=True):
        const_q = tf.Variable(tf.ones(shape=(), dtype=tf.int32), name="const_q")
        const_q = tf.sqrt(tf.square(tf.to_float(tf.to_float(const_q))))
        c_q = tf.scalar_mul(tf.to_float(const_q),q)
        q_sq = tf.square(q,name="q_sq")
        const_q_sq = tf.Variable(tf.ones(shape=(), dtype=tf.int32), name="const_q_sq")
        const_q_sq = tf.sqrt(tf.square(tf.to_float(tf.to_float(const_q_sq))))
        c_q_sq = tf.scalar_mul(tf.to_float(const_q_sq),q_sq)
        q_sum = tf.math.add(c_q,c_q_sq)
        q_1 = tf.stack([c_q,c_q_sq,q_sum])
        print("q1:",q_1.get_shape().as_list())
#     with tf.variable_scope("pq", reuse=True):
        pq_1 = tf.concat([p_1,q_1],0,name="pq_1")
        print("pq1:",pq_1.get_shape().as_list())
        # Binary operation
        num_row = pq_1.get_shape().as_list()[0]
        sum_pq = tf.zeros(shape=(),dtype=tf.float32)
        prod_pq = tf.zeros(shape=(),dtype=tf.float32)
        counter = 0
        for i in range(num_row):
            for j in range(i,num_row):
                if counter == 0:
                    sum_pq = tf.math.add(tf.expand_dims(pq_1[i],0),tf.expand_dims(pq_1[j],0))
                    prod_pq = tf.multiply(tf.expand_dims(pq_1[i],0),tf.expand_dims(pq_1[j],0))
                    print("pq_1[0]:",pq_1[i].get_shape().as_list())
                    print("sum_pq init:",sum_pq.get_shape().as_list())
                    print("prod_pq init:",prod_pq.get_shape().as_list())
                else:
                    sum_pq = tf.concat([sum_pq, tf.math.add(tf.expand_dims(pq_1[i],0),tf.expand_dims(pq_1[j],0))],axis=0,name="sum_pq")
                    prod_pq = tf.concat([prod_pq, tf.multiply(tf.expand_dims(pq_1[i],0),tf.expand_dims(pq_1[j],0))],axis=0,name="prod_pq")
                    tf.expand_dims(sum_pq,0)
                    tf.expand_dims(prod_pq,0)
                counter += 1
        print("sum_pq:",sum_pq.get_shape().as_list())
        print("prod_pq:",prod_pq.get_shape().as_list())
        # Log
        log_pq_1 = tf.math.log(pq_1)
        log_sum_pq = tf.math.log(sum_pq)
        const_sum_log = tf.Variable(tf.ones(shape=(), dtype=tf.int32), name="const_sum_log")
        const_sum_log = tf.multiply(tf.sqrt(tf.square(tf.to_float(const_sum_log))),-1)
        log_sum_pq = tf.multiply(log_sum_pq, tf.to_float(const_sum_log))
        log_prod_pq = tf.math.log(prod_pq)
        const_prod_log = tf.Variable(tf.ones(shape=(), dtype=tf.int32), name="const_prod_log")
        const_prod_log = tf.multiply(tf.sqrt(tf.square(tf.to_float(const_prod_log))),-1)
        log_prod_pq = tf.divide(log_prod_pq, tf.to_float(const_prod_log))
        log_pq = tf.concat([log_pq_1,log_sum_pq,log_prod_pq],0,name="log_pq")
        print("log_pq:",log_pq.get_shape().as_list())

#         log_p = tf.math.log(p_1)
#         log_q = tf.math.log(q_1)
#         log_pq = tf.concat([log_p,log_q],0,name="log_pq")
#         print("log_pq:",log_pq.get_shape().as_list())
        # Compare
        
        W = tf.Variable(tf.zeros([1,log_pq.get_shape().as_list()[0]], dtype=tf.float32), name="W")
        W = tf.where(tf.is_nan(W), tf.zeros_like(W), W)
        vol = tf.matmul(W,log_pq, name="vol")
        print("W:",W.get_shape().as_list())
        print("predicted volume:",vol.get_shape().as_list())
        vol_mask = tf.logical_not(tf.math.logical_or(tf.is_inf(vol),tf.is_nan(vol)))
        input_vol_mask = tf.boolean_mask(input_volume, vol_mask)
        pred_vol_mask = tf.boolean_mask(vol, vol_mask)
        dist = tf.pow(input_vol_mask-pred_vol_mask, 2)
#         max_cost = 0
#         cost_dist = tf.cond(tf.math.logical_or(tf.is_inf(dist),tf.is_nan(dist)), lambda:tf.constant(max_cost,dtype=tf.float32), lambda:dist)
        cost_op = tf.math.add(tf.reduce_mean(dist),tf.abs(1.0-tf.reduce_sum(W)))
        cost_op = tf.cond(tf.math.logical_or(tf.is_inf(cost_op),tf.is_nan(cost_op)), lambda:tf.constant(sys.maxsize,dtype=tf.float32), lambda:cost_op)
        print("cost:",cost_op.get_shape().as_list())
    return vol,cost_op,input_p,input_volume,pq_1,sum_pq,prod_pq,input_vol_mask,pred_vol_mask,vol_mask,W

def train_one_epoch(sess,train_op,cost_op,num_batches,batch_size,input_p,input_param,input_volume,input_vol,vol, input_vol_mask, pred_vol_mask, vol_mask,pq_1,sum_pq,prod_pq,W):
    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = (batch_idx+1) * batch_size
        param_train = input_param[start_idx:end_idx].reshape([2,batch_size])
        vol_train = input_vol[start_idx:end_idx].reshape([1,batch_size])
        vol_pred,_,batch_cost,sess_vol,sess_vol_pred,sess_mask,sess_pq_1,sess_sum_pq,sess_prod_pq,sess_W = sess.run([vol,train_op,cost_op,input_vol_mask,pred_vol_mask, vol_mask,pq_1,sum_pq,prod_pq,W],feed_dict={input_p: param_train, input_volume: vol_train})
#         print("pq_1: ",sess_pq_1)
#         print("sum_pq: ",sess_sum_pq)
#         print("prod_pq: ",sess_prod_pq)
#         print("W: ",sess_W)
#         print("predicted volume: ", vol_pred)
#         print("mask: ", sess_mask)
#         print("train volume masked: ",sess_vol)
#         print("predicted volume masked: ",sess_vol_pred)
        print("loss: ", batch_cost)
        
def eval_one_epoch(sess, cost_op, num_test_batches, input_p, test_param, input_volume, test_vol):
    for batch_idx in range(num_test_batches):
        start_idx = batch_idx * batch_size
        end_idx = (batch_idx+1) * batch_size
        input_param = test_param[start_idx:end_idx].reshape([2,batch_size])
        input_vol = test_vol[start_idx:end_idx].reshape([1,batch_size])
        cost = sess.run(cost_op, feed_dict={input_p: input_param, input_volume: input_vol})
        print("test loss: ", cost)

def generate_data(max_h):
    p_train = 0.75 # precent of training set of total generated set
    vol_data = []
    for i in range(1,max_h):
        for j in range(0,i):
            vol_data.append([i,j,np.log(func(i,j))])
    shuffle(vol_data)
    num_data = len(vol_data)
    num_train = int(p_train*num_data)
    train_data = vol_data[:num_train]
    test_data = vol_data[num_train+1:]
    train = []
    train_vol = []
    test = []
    test_vol = []
    for group in train_data:
        train.append([group[0],group[1]])
        train_vol.append(group[2])
    for group in test_data:
        test.append([group[0],group[1]])
        test_vol.append(group[2])
    return train, train_vol, test, test_vol

def train(train,train_vol,test,test_vol,param_name):
    tf.reset_default_graph()
    num_train = len(train)
    num_test = len(test)
    num_param = len(train[0])
    batch_size = 10
    num_batches = int(num_train/batch_size)
    num_test_batches = int(num_test/batch_size)
    print('number of training set: ', num_train," number of test set: ", num_test)
    print('number of parameters: ', num_param)
    print('batch_size: ', batch_size)
    print('number of batches: ', num_batches)
    input_param = np.array(train)
    input_vol = np.array(train_vol)
    test_param = np.array(test)
    test_vol = np.array(test_vol)
    vol,cost_op,input_p,input_volume,pq_1,sum_pq,prod_pq,input_vol_mask,pred_vol_mask,vol_mask,W = model(num_param, batch_size)
    learning_rate = tf.Variable(0.5, trainable=True)
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_op = optimizer.minimize(cost_op)
    sess = tf.Session() # Create TensorFlow session
    saver = tf.train.Saver() # Save Tensorflow graph
    print ("Beginning Training")
    with sess.as_default():
        collection = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='pq')
        init = tf.global_variables_initializer()
        sess.run(init)
        sess.run(tf.assign(learning_rate, alpha))
        for epoch in range(1, max_epochs):
            print('epoch: ', epoch)
            train_one_epoch(sess,train_op,cost_op,num_batches,batch_size,input_p,input_param,input_volume,input_vol,vol,input_vol_mask,pred_vol_mask,vol_mask,pq_1,sum_pq,prod_pq,W)
            eval_one_epoch(sess, cost_op, num_test_batches, input_p, test_param, input_volume, test_vol)
        print('Finished training.')
        for x in collection:
            print("collection: ", x.name)
            if x.name in param_name:
                print("name: ", x.name)
                print("variables: ", sess.run(x))
        

print("Done.")

Done.


In [29]:
train_param, train_vol, test_param, test_vol = generate_data(100)
param_name = ["pq/const_sum_log:0","pq/const_prod_log:0","pq/log_pq:0",
              "pq/const_p_sq:0","pq/const_p_sq_1:0","pq/const_q:0",
              "pq/const_q_sq:0","pq/const_sum_log:0","pq/const_prod_log:0",
              "pq/W:0","pq/vol:0","pq/sum_pq:0","pq/prod_pq:0",
             "pq/W/Adam:0","pq/W/Adam_1:0","pq/Variable:0","pq/Variable_1:0",
             "pq/Variable_2:0","pq/Variable_3:0","pq/Variable_4:0",
             "pq/Variable_5:0"]
alpha = 0.5
last_cost = 0
batch_size = 10
max_epochs = 20
tolerance = 1e-3

train(train_param,train_vol,test_param,test_vol,param_name)

number of training set:  3712  number of test set:  1237
number of parameters:  2
batch_size:  10
number of batches:  371
input p: [2, 10]
input volume: [1, 10]
p: [10]
q: [10]
p1: [3, 10]
q1: [3, 10]
pq1: [6, 10]
pq_1[0]: [10]
sum_pq init: [1, 10]
prod_pq init: [1, 10]
sum_pq: [21, 10]
prod_pq: [21, 10]
log_pq: [48, 10]
W: [1, 48]
predicted volume: [1, 10]
cost: []
Beginning Training
epoch:  1
loss:  3.3499663
loss:  46255.816
loss:  2872.2576
loss:  9658.499
loss:  26332.543
loss:  20770.994
loss:  5903.6357
loss:  176.77278
loss:  8895.551
loss:  14956.835
loss:  10003.13
loss:  3152.8442
loss:  51.89039
loss:  1486.2998
loss:  5597.923
loss:  7283.387
loss:  4652.787
loss:  450.25482
loss:  58.084156
loss:  43.89303
loss:  342.30414
loss:  769.473
loss:  1027.496
loss:  991.2971
loss:  769.0479
loss:  359.98773
loss:  84.18832
loss:  2.331369
loss:  86.573204
loss:  190.28691
loss:  393.68692
loss:  579.8737
loss:  425.38095
loss:  164.48636
loss:  109.456635
loss:  8.524593
loss: 

loss:  3.495277
loss:  3.3440435
loss:  3.2947147
loss:  3.2641938
loss:  3.3471434
loss:  3.2631774
loss:  3.3363032
loss:  3.310501
loss:  3.4603066
loss:  3.3243601
loss:  3.2709572
loss:  3.311146
loss:  3.3838947
loss:  3.3270814
loss:  3.4956038
loss:  3.3214216
loss:  3.3939567
loss:  3.4664605
loss:  3.4745526
loss:  3.3876152
loss:  3.310738
loss:  3.2409391
loss:  3.3723447
loss:  3.2756102
loss:  3.4020715
loss:  3.4636085
loss:  3.4655554
loss:  3.3158119
loss:  3.408223
loss:  3.311223
loss:  3.3206475
loss:  3.4776752
loss:  3.4988797
loss:  3.1910863
loss:  3.3394694
loss:  3.3604066
loss:  3.4087272
loss:  3.488629
loss:  3.388274
loss:  3.2868547
loss:  3.3637953
loss:  3.219934
loss:  3.3291936
loss:  3.5267692
loss:  3.3890052
loss:  3.244212
loss:  3.4405322
loss:  3.437767
loss:  3.351717
loss:  3.2920132
loss:  3.5454154
loss:  3.3568978
loss:  3.3963485
loss:  3.2234275
loss:  3.129745
loss:  3.3878894
loss:  3.2181096
loss:  3.4292748
loss:  3.3924234
loss:  3.2

loss:  3.5454154
loss:  3.3568978
loss:  3.3963485
loss:  3.2234275
loss:  3.129745
loss:  3.3878894
loss:  3.2181096
loss:  3.4292748
loss:  3.3924234
loss:  3.292611
loss:  3.4494164
loss:  3.4221008
loss:  3.5052028
loss:  3.3093975
loss:  3.258627
loss:  3.333529
loss:  3.5024824
loss:  3.29678
loss:  3.4569948
loss:  3.3678436
loss:  3.184561
loss:  3.4446156
loss:  3.4246604
loss:  3.3429732
loss:  3.4529643
loss:  3.210472
loss:  3.1657853
loss:  3.403211
loss:  3.1913846
loss:  3.4133258
loss:  3.0302403
loss:  3.4257429
loss:  3.1943605
loss:  3.3769438
loss:  3.3973126
loss:  3.3968058
loss:  3.2524066
loss:  3.259178
loss:  3.2406738
loss:  3.3618884
loss:  3.4074295
loss:  3.4773564
loss:  3.5053656
loss:  3.4540741
loss:  3.5358896
loss:  3.304215
loss:  3.3967457
loss:  3.5472927
loss:  3.2479026
loss:  3.3936315
loss:  3.2983236
loss:  3.4142756
loss:  3.397585
loss:  3.3239365
loss:  3.4055839
loss:  3.1924994
loss:  3.3311188
loss:  3.2650893
loss:  3.3653193
loss:  3.

loss:  3.1924994
loss:  3.3311188
loss:  3.2650893
loss:  3.3653193
loss:  3.337768
loss:  3.2113194
loss:  3.360261
loss:  3.2990937
loss:  3.3698702
loss:  3.3079562
loss:  3.4538453
loss:  3.3807425
loss:  3.3969073
loss:  3.5076287
loss:  3.3906724
loss:  3.172385
loss:  3.331605
loss:  3.3328962
loss:  3.3243084
loss:  3.415894
loss:  3.199252
loss:  3.3782883
loss:  3.369969
loss:  3.123978
loss:  3.4005952
loss:  3.0782793
loss:  3.377177
loss:  3.4420993
loss:  3.2960489
loss:  3.2222173
loss:  3.3758855
loss:  3.3551161
loss:  3.3266284
loss:  3.1838303
loss:  3.221952
loss:  3.106844
loss:  3.3231952
loss:  3.4685645
loss:  3.315928
loss:  3.4550614
loss:  3.356509
loss:  3.3967345
loss:  3.3837583
loss:  3.2859626
loss:  3.4083812
loss:  3.3724055
loss:  3.3712602
loss:  3.5044312
loss:  3.4397297
loss:  3.1538196
loss:  3.06938
loss:  3.4225183
loss:  3.3694413
loss:  3.308172
loss:  3.4029553
loss:  3.3760357
loss:  3.2572553
loss:  3.3576384
loss:  3.3736107
loss:  3.1387

loss:  3.4029553
loss:  3.3760357
loss:  3.2572553
loss:  3.3576384
loss:  3.3736107
loss:  3.1387205
loss:  3.4290042
loss:  3.3738148
loss:  3.2640693
loss:  3.395864
loss:  3.3579502
loss:  3.4470286
loss:  3.5544333
loss:  2.9859958
loss:  3.4336255
loss:  3.3293347
loss:  3.2768402
loss:  3.1563442
loss:  3.2294176
loss:  3.3910003
loss:  3.433412
loss:  3.2366717
loss:  3.1825843
loss:  3.0991313
loss:  3.5137153
loss:  3.3659663
loss:  3.3761287
loss:  3.2482219
loss:  3.248961
loss:  3.285802
loss:  3.3399036
loss:  3.1397197
loss:  3.337892
loss:  3.3337886
loss:  3.4626777
loss:  3.4003894
loss:  3.4597917
loss:  3.457478
loss:  3.1750581
loss:  3.4105167
loss:  3.313811
loss:  3.412146
loss:  3.4789968
loss:  3.3765457
loss:  3.1980202
loss:  3.4611795
loss:  3.2521904
loss:  3.499931
loss:  3.2146163
loss:  3.509575
loss:  3.4588335
loss:  3.398045
loss:  3.2072139
loss:  3.4658751
loss:  3.4479752
loss:  3.3457494
loss:  3.3534482
loss:  3.4647584
loss:  3.5458825
loss:  3

...

loss:  3.3231952
loss:  3.4685645
loss:  3.315928
loss:  3.4550614
loss:  3.356509
loss:  3.3967345
loss:  3.3837583
loss:  3.2859626
loss:  3.4083812
loss:  3.3724055
loss:  3.3712602
loss:  3.5044312
loss:  3.4397297
loss:  3.1538196
loss:  3.06938
loss:  3.4225183
loss:  3.3694413
loss:  3.308172
loss:  3.4029553
loss:  3.3760357
loss:  3.2572553
loss:  3.3576384
loss:  3.3736107
loss:  3.1387205
loss:  3.4290042
loss:  3.3738148
loss:  3.2640693
loss:  3.395864
loss:  3.3579502
loss:  3.4470286
loss:  3.5544333
loss:  2.9859958
loss:  3.4336255
loss:  3.3293347
loss:  3.2768402
loss:  3.1563442
loss:  3.2294176
loss:  3.3910003
loss:  3.433412
loss:  3.2366717
loss:  3.1825843
loss:  3.0991313
loss:  3.5137153
loss:  3.3659663
loss:  3.3761287
loss:  3.2482219
loss:  3.248961
loss:  3.285802
loss:  3.3399036
loss:  3.1397197
loss:  3.337892
loss:  3.3337886
loss:  3.4626777
loss:  3.4003894
loss:  3.4597917
loss:  3.457478
loss:  3.1750581
loss:  3.4105167
loss:  3.313811
loss:  3.

loss:  3.337892
loss:  3.3337886
loss:  3.4626777
loss:  3.4003894
loss:  3.4597917
loss:  3.457478
loss:  3.1750581
loss:  3.4105167
loss:  3.313811
loss:  3.412146
loss:  3.4789968
loss:  3.3765457
loss:  3.1980202
loss:  3.4611795
loss:  3.2521904
loss:  3.499931
loss:  3.2146163
loss:  3.509575
loss:  3.4588335
loss:  3.398045
loss:  3.2072139
loss:  3.4658751
loss:  3.4479752
loss:  3.3457494
loss:  3.3534482
loss:  3.4647584
loss:  3.5458825
loss:  3.5064764
loss:  3.4073498
loss:  3.352224
loss:  3.1683564
loss:  3.3441565
loss:  3.3531632
loss:  3.1665053
loss:  3.5209334
loss:  3.2162437
loss:  3.4523857
loss:  3.480699
loss:  3.405087
loss:  3.2257304
loss:  3.3596225
loss:  3.2550564
loss:  3.3307655
loss:  3.3597944
loss:  3.3838365
loss:  3.3275306
loss:  3.4962714
loss:  3.2498958
loss:  3.2216232
test loss:  3.2779408
test loss:  3.3741226
test loss:  3.4579062
test loss:  3.4431853
test loss:  3.2131143
test loss:  3.1592467
test loss:  3.3513901
test loss:  3.5809174
t

loss:  3.405087
loss:  3.2257304
loss:  3.3596225
loss:  3.2550564
loss:  3.3307655
loss:  3.3597944
loss:  3.3838365
loss:  3.3275306
loss:  3.4962714
loss:  3.2498958
loss:  3.2216232
test loss:  3.2779408
test loss:  3.3741226
test loss:  3.4579062
test loss:  3.4431853
test loss:  3.2131143
test loss:  3.1592467
test loss:  3.3513901
test loss:  3.5809174
test loss:  3.366503
test loss:  3.4219198
test loss:  3.5057216
test loss:  3.3006897
test loss:  3.2383645
test loss:  3.3109999
test loss:  3.3405442
test loss:  3.3430638
test loss:  3.0451152
test loss:  3.0598516
test loss:  3.3721128
test loss:  3.265277
test loss:  3.3180232
test loss:  3.4070864
test loss:  3.1891122
test loss:  3.4102585
test loss:  3.2255607
test loss:  3.3778598
test loss:  3.2764993
test loss:  3.4140506
test loss:  3.286155
test loss:  3.4306352
test loss:  3.3137853
test loss:  3.286738
test loss:  3.3820584
test loss:  3.3086495
test loss:  3.345379
test loss:  3.2801359
test loss:  3.575767
test l

test loss:  3.2783096
test loss:  3.3306935
test loss:  3.4595952
test loss:  3.246283
test loss:  3.4085248
test loss:  3.1925888
test loss:  3.571927
test loss:  3.5444736
test loss:  3.3443341
test loss:  3.5025468
test loss:  3.406177
test loss:  3.097153
test loss:  3.3892965
test loss:  3.4678013
test loss:  3.3574328
test loss:  2.9388096
test loss:  3.3264244
test loss:  3.3542533
test loss:  3.0519133
test loss:  3.2691605
test loss:  3.1182137
test loss:  3.3762345
test loss:  3.2976096
test loss:  3.5154572
test loss:  3.4935286
test loss:  3.3871555
test loss:  3.3930771
test loss:  3.4247742
test loss:  3.362335
test loss:  3.4383729
test loss:  3.2477252
test loss:  3.384646
test loss:  3.3140547
test loss:  3.3000216
test loss:  3.3697617
test loss:  3.1192145
test loss:  3.3322744
test loss:  3.327304
test loss:  3.2491233
test loss:  3.4148648
test loss:  3.4640822
test loss:  3.434976
test loss:  3.237121
test loss:  3.310526
test loss:  3.4556427
test loss:  3.390537

variables:  1
collection:  pq/const_q:0
name:  pq/const_q:0
variables:  1
collection:  pq/const_q_sq:0
name:  pq/const_q_sq:0
variables:  1
collection:  pq/const_sum_log:0
name:  pq/const_sum_log:0
variables:  1
collection:  pq/const_prod_log:0
name:  pq/const_prod_log:0
variables:  1
collection:  pq/W:0
name:  pq/W:0
variables:  [[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan]]
collection:  pq/W/Adam:0
name:  pq/W/Adam:0
variables:  [[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan]]
collection:  pq/W/Adam_1:0
name:  pq/W/Adam_1:0
variables:  [[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  na